# Energy Storage Dispatch Algorithm
In this notebook, we'll lay out the algorithm for our **energy storage dispatch** in commercial buildings.

**Key variables:**
* Load Profile
* Power Prices
* Battery Characteristics

**Key outputs:**
* Price Savings

From which we'll derive -
* Dispatch Characteristics
* Modified Load Shape



# Power Prices from ERCOT

In this section we'll be extracting power prices from ERCOT's 2017 power prices. We'll use HB_BUSAVG for now (to be changed later).



In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

df_powerprices = pd.read_csv(r'C:\Users\Niel Patel\Documents\GitHub\commercial_storage_analysis\data\ERCOT_Power_Prices.csv')
df_powerprices.columns = ['Delivery_Date','Hour_Ending','Repeated_Hour_Flag','Settlement_Point','Settlement_Point_Price']
df_powerprices.head()

df_AustinPP = df_powerprices[df_powerprices.Settlement_Point == 'HB_BUSAVG']

df_AustinPP = df_AustinPP.reset_index(drop=True)
df_AustinPP.head()

,Delivery_Date,Hour_Ending,Repeated_Hour_Flag,Settlement_Point,Settlement_Point_Price
0,01/01/2017,01:00,N,HB_BUSAVG,23.23
1,01/01/2017,02:00,N,HB_BUSAVG,21.82
2,01/01/2017,03:00,N,HB_BUSAVG,20.16
3,01/01/2017,04:00,N,HB_BUSAVG,20.00
4,01/01/2017,05:00,N,HB_BUSAVG,20.24


# Combining Data

Here we'll initialize the model and read all the data into the data frame. Combine Power Prices and Load.

I've multiplied Wholesale Power Prices by 3 so make it more realistic - to be changed.


In [31]:


#Read Load Forecast
df_load = pd.read_csv(r'C:\Users\Niel Patel\Documents\GitHub\commercial_storage_analysis\data\Load_Price.csv')
df_load["Power Price ($/kWh)"] = df_AustinPP["Settlement_Point_Price"].astype(float)*3/1000
df_load["Battery_Max_Capacity"]= 1000
df_load["Battery_Current_Charge"]=0
df_load["Battery_Load"] = 0
df_load["Facility_Net_Load"] = df_load["Facility Load Data, pre-storage/renewables (kW)"]+df_load["Battery_Load"]
df_load["Pre-storage_Price"] = df_load["Facility Load Data, pre-storage/renewables (kW)"]*df_load["Power Price ($/kWh)"]
df_load["Post-storage_Price"] = df_load["Facility_Net_Load"]*df_load["Power Price ($/kWh)"]
df_load.head()




,Date,Time,"Facility Load Data, pre-storage/renewables (kW)",Power Price ($/kWh),Battery_Max_Capacity,Battery_Current_Charge,Battery_Load,Facility_Net_Load,Pre-storage_Price,Post-storage_Price
0,01/01/2017,1:00,2388,0.06969,1000,0,0,2388,166.41972,166.41972
1,01/01/2017,2:00,1992,0.06546,1000,0,0,1992,130.39632,130.39632
2,01/01/2017,3:00,2196,0.06048,1000,0,0,2196,132.81408,132.81408
3,01/01/2017,4:00,1744,0.06000,1000,0,0,1744,104.64000,104.64000
4,01/01/2017,5:00,2384,0.06072,1000,0,0,2384,144.75648,144.75648


# Combining Data and Dispatch
In this section we'll be dipatching the battery to minimize the cost to the building.

**Battery Decision Options**
Charge
Discharge

**Logic**
To be implemented in the future - 
Minimize Load(n) x PowerPrice(n) + Load(n+1) x PowerPrice(n+1) + Load (n+2) x PowerPrice(n+2)

Such that
Battery Charge <= Maximum Battery Capacity
Battery Charge >= 0


In [40]:
count = 0
last_pp = 0.10
last_charge = 0
for index, row in df_load.iterrows():
    now_pp = row["Power Price ($/kWh)"]
    
    #Loop for charging/discharging battery
    
    if now_pp < last_pp:
        
        #charge
        
        if last_charge >= 0 and last_charge < 1000:
            df_load.at[count,"Battery_Load"]= 1000 - last_charge
        df_load.at[count,"Battery_Current_Charge"] = 1000
        last_charge = df_load.at[count,"Battery_Current_Charge"]

    else:
        
        #discharge
        
        if last_charge > 0 and last_charge <= 1000:
            df_load.at[count,"Battery_Load"]= -last_charge
        df_load.at[count,"Battery_Current_Charge"] = 0
        last_charge = df_load.at[count,"Battery_Current_Charge"]
        
    last_pp = row["Power Price ($/kWh)"]
    count += 1
    
#Calculating Net Load
    
df_load["Facility_Net_Load"] = df_load["Facility Load Data, pre-storage/renewables (kW)"]+df_load["Battery_Load"]

#Calculating Net Price
df_load["Post-storage_Price"] = df_load["Facility_Net_Load"]*df_load["Power Price ($/kWh)"]

Total_Price_Prestorage = df_load["Pre-storage_Price"].sum()
Total_Load_Prestorage = df_load["Facility Load Data, pre-storage/renewables (kW)"].sum()

Avg_Price_Prestorage = Total_Price_Prestorage/Total_Load_Prestorage

print('The total pre-storage load is', Total_Load_Prestorage,'and total pre-storage price is',Total_Price_Prestorage,'. And average price is',Avg_Price_Prestorage)

Total_Price_Poststorage = df_load["Post-storage_Price"].sum()
Total_Load_Poststorage = df_load["Facility_Net_Load"].sum()

Avg_Price_Poststorage = Total_Price_Poststorage/(Total_Load_Poststorage+last_charge)
print('The total post-storage load is', Total_Load_Poststorage,'and total post-storage price is',Total_Price_Poststorage,'. And average price is',Avg_Price_Poststorage)


The total pre-storage load is 23500890 and total pre-storage price is 1768775.19882 . And average price is 0.07526417930640074
The total post-storage load is 23501890 and total post-storage price is 1788358.2688199999 . And average price is 0.07609099429134034
